In [ ]:
from cheetah import Segment, ParticleBeam
import torch
import torch.nn as nn
import torch.nn.functional as F
import epics
import os
import numpy as np
import pprint
import h5py
from typing import List, Dict, Any, Callable
import yaml
import matplotlib.pyplot as plt
import math

In [ ]:
def extract_group_contents(group, exclude_keys=None):
    """Safely extract datasets from a group, handling scalars and nested groups."""
    if exclude_keys is None:
        exclude_keys = set()
    data = {}
    for key in group:
        if key in exclude_keys:
            continue
        item = group[key]
        if isinstance(item, h5py.Dataset):
            data[key] = item[()] if item.shape == () else item[:]
        elif isinstance(item, h5py.Group):
            data[key] = extract_group_contents(item, exclude_keys)
    return data


images = {}
with h5py.File('../h5/tcav_calibration.h5', 'r') as f:
    image_root = f['images']
    exclude_keys = {'raw_images','processed_images','total_intensities', 'signal_to_noise_ratios'}

    for image_key in image_root:
        image_group = image_root[image_key]
        image_data = extract_group_contents(image_group, exclude_keys=exclude_keys)
        images[image_key] = image_data



In [ ]:
# Since all the quad settings are matched in the simulated server and calibration file the only PVs we care about is tcav AREQ, PREQ
# Can extract these values from the group contents pretty easily
# We will also plot the average beam size x measurement versus phase

keys = [key for key in images.keys()]
centroid_stack = np.stack([
    np.nanmean(images[key]['centroids'],axis=0) for key in keys])
phase_stack = np.stack([
    images[key]['TCAV:DIAG0:11:PREQ'] for key in images])
phase_stack = phase_stack * (math.pi/180)
print(phase_stack)
print(centroid_stack)


In [ ]:
from scipy.optimize import curve_fit
X= phase_stack
Y= centroid_stack
# 1. Linear fit for centroid X
def linear(x, m, b):
    return m * x + b

params_x, _ = curve_fit(linear, X, Y[:, 0])
fit_x = linear(X, *params_x)

# 2. Sine fit for centroid Y (still using degrees)
def sine_deg(x_deg, amplitude, phase_shift, frequency, offset):
    x_rad = np.radians(x_deg)
    return amplitude * np.sin(frequency * x_rad + phase_shift) + offset

p0_y = [np.ptp(Y[:, 1]) / 2, 0.0, 1.0, np.mean(Y[:, 1])]
params_y, _ = curve_fit(sine_deg, X, Y[:, 1], p0=p0_y)
fit_y = sine_deg(X, *params_y)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Centroid X (Linear)
axes[0].scatter(X, Y[:, 0], color='red', label='Centroid X')
axes[0].plot(X, fit_x, color='black', linestyle='--', label='Linear Fit')
axes[0].set_title('Centroid X vs Phase (Linear)')
axes[0].set_xlabel('Phase [deg]')
axes[0].set_ylabel('Centroid X')
axes[0].legend()
axes[0].grid(True)
# Suppose params_x = [m, b]
eqn_x = f"y = {params_x[0]:.3f}·x + {params_x[1]:.3f}"
axes[0].text(0.05, 0.95, eqn_x, transform=axes[0].transAxes,
             fontsize=12, verticalalignment='bottom', bbox=dict(facecolor='white', alpha=0.7))

# Centroid Y (Sine)
axes[1].scatter(X, Y[:, 1], color='green', label='Centroid Y')
axes[1].plot(X, fit_y, color='black', linestyle='--', label='Sine Fit')
axes[1].set_title('Centroid Y vs Phase (Sine)')
axes[1].set_xlabel('Phase [deg]')
axes[1].set_ylabel('Centroid Y')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.show()

In [ ]:
from lcls_tools.common.devices.reader import create_screen, create_tcav, create_magnet
from ml_tto.automatic_emittance.screen_profile import ScreenBeamProfileMeasurement
from ml_tto.automatic_emittance.image_projection_fit import RecursiveImageProjectionFit
from ml_tto.automatic_emittance.plotting import plot_image_projection_fit

In [ ]:
def show_side_by_side(img1: np.ndarray, img2: np.ndarray, titles=('Original', 'Noisy'), cmap='hot'):
    """
    Plot two 2D images side-by-side as heat maps.

    Parameters:
        img1, img2 (np.ndarray): 2D arrays to plot.
        titles (tuple): Titles for the subplots.
        cmap (str): Colormap to use.
    """
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    for ax, img, title in zip(axes, [img1, img2], titles):
        im = ax.imshow(img, cmap=cmap, origin='lower', aspect='auto')
        ax.set_title(title)
        ax.set_xlabel('X pixels')
        ax.set_ylabel('Y pixels')
        fig.colorbar(im, ax=ax, shrink=0.75, label='Intensity')
    plt.tight_layout()
    plt.show()

In [ ]:
#Start server, track screen, fit beam get result, vary phase repeat for each phase in phase_stack, use lcls_tools library to control
# the screen, tcav, and magnets if needed
#TODO: fix tcav.phase_set not fucking working
screen = create_screen('DIAG0', 'OTRDG02')
#tcav = create_tcav('DIAG0', 'TCXDG0')
'''
images = []
image =  screen.image 
noise_std = 0.2 * np.max(image)
image_noisey = image + np.abs(np.random.normal(loc=0, scale=noise_std, size=image.shape))
show_side_by_side(image,image_noisey)
'''
i= 0
for phase in phase_stack:
    epics.caput('TCAV:DIAG0:11:PREQ', phase)
    image =  screen.image
    noise_std = 0.2 * (np.max(image) + .0001)
    #image += np.abs(np.random.normal(loc=0, scale= noise_std , size=image.shape))
    if i ==0:
        sneaky = image
    i+=1
    images.append(image)



In [ ]:
#Fit images get centroids
#bad fits... eed to get centroids some other way
recImg = RecursiveImageProjectionFit(visualize=True)
sim_centroids = []
for image in images:
    result = recImg._fit_image(image)
    sim_centroids.append(result.centroid)
    #Dump images to file for training set.

sim_centroids_stack = np.stack([centroid for centroid in sim_centroids])
#plt.imshow(sneaky, cmap='hot', origin='lower', aspect='auto')

In [ ]:

#TODO: figure out what the problem with the fits is? for now interpolate points and try and optimize with a few interpolated points
c = torch.tensor(sim_centroids)
c

In [ ]:


def interpolate_nans(data):
    """
    Linearly interpolates missing values (np.nan) in a 1D NumPy array or PyTorch tensor.
    Extrapolates linearly at the ends if NaNs are present there.
    
    Parameters:
        data (np.ndarray or torch.Tensor): 1D array/tensor with NaNs.

    Returns:
        torch.Tensor: 1D tensor with NaNs filled.
    """
    # Convert to NumPy array if torch tensor
    is_tensor = isinstance(data, torch.Tensor)
    arr = data.numpy() if is_tensor else np.array(data, dtype=float)

    x = np.arange(len(arr))
    mask = np.isnan(arr)

    if np.all(mask):
        raise ValueError("All values are NaN — cannot interpolate.")

    known_x = x[~mask]
    known_y = arr[~mask]

    # Interpolate internal NaNs
    arr[mask] = np.interp(x[mask], known_x, known_y)

    # Extrapolate leading NaNs
    if known_x[0] > 0:
        slope_start = (known_y[1] - known_y[0]) / (known_x[1] - known_x[0])
        for i in range(known_x[0]):
            arr[i] = known_y[0] - (known_x[0] - i) * slope_start

    # Extrapolate trailing NaNs
    if known_x[-1] < len(arr) - 1:
        slope_end = (known_y[-1] - known_y[-2]) / (known_x[-1] - known_x[-2])
        for i in range(known_x[-1] + 1, len(arr)):
            arr[i] = known_y[-1] + (i - known_x[-1]) * slope_end

    return torch.tensor(arr) if is_tensor else arr

In [ ]:
#In all the times i ran the program this was the best set of fits so I felt liek it was worth keeping and interoplating
best_fits= torch.tensor([np.nan, 802.31841187, 758.54816323, 716.24749925,
       685.01428837, 653.7810775 , 625.62393425, 598.56945705,
       np.nan, np.nan])
interpolated = interpolate_nans(best_fits)
interpolated2 = interpolate_nans(c[:,0])

print(interpolated)
print(interpolated2)

In [ ]:
# find fit of sim data
params_x_sim, _ = curve_fit(linear, X, interpolated2)
fit_x_sim = linear(X, *params_x_sim)

In [ ]:


fig, axes = plt.subplots( figsize=(12, 4))

# Centroid X (Linear)
axes.scatter(X, Y[:, 0], color='red', label='Centroid X')
axes.scatter(X,interpolated2, color= 'green', label = 'Centroid X Sim')
axes.plot(X, fit_x, color='black', linestyle='--', label='Experiment Fit')
axes.plot(X, fit_x_sim, color='blue', linestyle='--', label='Sim Fit')
axes.set_title('Centroid X vs Phase (Linear)')
axes.set_xlabel('Phase [deg]')
axes.set_ylabel('Centroid X')
axes.legend()
axes.grid(True)
# Suppose params_x = [m, b]
eqn_x = f"y = {params_x[0]:.3f}·x + {params_x[1]:.3f}"
axes.text(0.05, 0.95, eqn_x, transform=axes.transAxes,
             fontsize=12, verticalalignment='bottom', bbox=dict(facecolor='white', alpha=0.7))


plt.tight_layout()
plt.show()

In [ ]:
voltage_in_MV = epics.caget('TCAV:DIAG0:11:AREQ', use_monitor=False)
print(f'before {voltage_in_MV}')
voltage_in_MV = voltage_in_MV + .01
epics.caput('TCAV:DIAG0:11:AREQ', voltage_in_MV)
voltage_in_MV = epics.caget('TCAV:DIAG0:11:AREQ', use_monitor=False)
print(f'after {voltage_in_MV}')

In [ ]:


#ignore this is from cheetah examples
def train(num_steps: int, target_beam_parameters: torch.Tensor, lr=0.1) -> dict:
    beam_parameter_history = []
    magnet_setting_history = []
    loss_history = []

    optimizer = torch.optim.Adam(ares_ea.parameters(), lr=lr)

    for _ in range(num_steps):
        optimizer.zero_grad()

        outgoing_beam = ares_ea.track(incoming_beam)

        observed_beam_parameters = torch.stack(
            [
                outgoing_beam.mu_x,
                outgoing_beam.sigma_x,
                outgoing_beam.mu_y,
                outgoing_beam.sigma_y,
            ]
        )
        loss = F.mse_loss(observed_beam_parameters, target_beam_parameters)

        loss.backward()

        # Log magnet settings and beam parameters
        loss_history.append(loss.item())
        beam_parameter_history.append(observed_beam_parameters.detach().numpy())
        magnet_setting_history.append(
            torch.stack(
                [
                    ares_ea.AREAMQZM1.k1,
                    ares_ea.AREAMQZM2.k1,
                    ares_ea.AREAMCVM1.angle,
                    ares_ea.AREAMQZM3.k1,
                    ares_ea.AREAMCHM1.angle,
                ]
            )
            .detach()
            .numpy()
        )
        optimizer.step()

    history = {
        "loss": loss_history,
        "beam_parameters": beam_parameter_history,
        "magnet_settings": magnet_setting_history,
    }
    return history
 
history = train(num_steps=100, target_beam_parameters=torch.zeros(4))



In [ ]:
# how is the amplitude related to the slope of the curve? 
def train(num_steps:int, target_beam_parameters: epics.PV, lr = .0001) -> dict:
    # working in single case for now.
    beam_parameter_history = []
    loss_history = []
    amplitude = epics.caget('TCAV:DIAG0:11:AREQ', use_monitor=False)
